In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image


learning_rate = 0.001
training_epochs = 15
batch_size = 100


# mnist data

transform = transforms.ToTensor()
train_mnist = dsets.MNIST(root='./data', train=True, download=True, transform=transform)
test_mnist = dsets.MNIST(root='./data', train=False, download=True, transform=transform)
train_dataloader = DataLoader(train_mnist, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_mnist, batch_size=batch_size, shuffle=True, drop_last=True)


# device

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)


# CNN

class CNN(nn.Module) :

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# training

training_batch = len(train_dataloader)

for epoch in range(training_epochs):
    avg_loss = 0
    step = 0

    for inputs, labels in train_dataloader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        avg_loss += loss/training_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_loss))


# test

avg_loss_test = 0
test_batch = len(test_dataloader)
total = 0
correct = 0

with torch.no_grad():
    for inputs_test, labels_test in test_dataloader:

        inputs_test = inputs_test.to(device)
        labels_test = labels_test.to(device)
        outputs_test = model(inputs_test)

        loss_test = criterion(outputs_test, labels_test)
        avg_loss_test+= loss_test/test_batch

        _, predicted = torch.max(outputs_test.data, 1)
        for label, prediction in zip(labels_test, predicted):
            if label == prediction:
                correct += 1
            total += 1

accuracy = 0
if total != 0:
    accuracy = correct / total

print('test loss : %f'%avg_loss_test)
print('accuracy : %f'%accuracy)

step : 600
[Epoch:    1] cost = 0.25722912
step : 600
[Epoch:    2] cost = 0.0922976956
step : 600
[Epoch:    3] cost = 0.0710320845
step : 600
[Epoch:    4] cost = 0.0568704121
step : 600
[Epoch:    5] cost = 0.0500989109
step : 600
[Epoch:    6] cost = 0.0442770496
step : 600
[Epoch:    7] cost = 0.0391521119
step : 600
[Epoch:    8] cost = 0.0342525765
step : 600
[Epoch:    9] cost = 0.0323464759
step : 600
[Epoch:   10] cost = 0.0296998117
step : 600
[Epoch:   11] cost = 0.0270394441
step : 600
[Epoch:   12] cost = 0.0263590105
step : 600
[Epoch:   13] cost = 0.023350792
step : 600
[Epoch:   14] cost = 0.0224301815
step : 600
[Epoch:   15] cost = 0.019308826
test loss : 0.053236
accuracy : 0.986900
